In [49]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

In [50]:
#PCSI0072

In [51]:
#Read the data

GenomicFeatures_PCSI0072 = pd.read_csv(open('data/PCSI0072/GenomicFeatures_PCSI0072.txt','r'),sep='\t')
SNVCalls_PCSI0072    = pd.read_csv(open('data/PCSI0072/SNVCalls_PCSI0072.txt','r'),sep='\t') 

In [52]:
print GenomicFeatures_PCSI0072.shape
print SNVCalls_PCSI0072.shape

(83808, 15)
(83808, 136)


In [53]:
np.sum(GenomicFeatures_PCSI0072.isnull())

CHROM                  0
POS                    0
RefAllele              0
NonRefAllele           0
BaseQual               0
TumourCoverage         0
NormalCoverage         0
MapQual                0
ReadPosition           6
Trinucleotide          0
HomopolymerRate        0
GCcontent              0
GenomicLocation    25430
DistanceSNP            0
StrandBias             6
dtype: int64

In [54]:
GenomicFeatures_PCSI0072['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_PCSI0072['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_PCSI0023.isnull())

In [55]:
np.unique(GenomicFeatures_PCSI0072['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_exonic',
       'ncRNA_intronic', 'splicing', 'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [56]:
GenomicFeatures_PCSI0072.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10180,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793
1,chr1,10352,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667
2,chr1,16257,69,32,3047,106,76,16,67,CCC,2.333333,0.517413,ncRNA_intronic,35981,0.556604
3,chr1,17594,95,19,3588,120,153,0,60,CCT,2.605634,0.631841,ncRNA_intronic,34644,0.500000
4,chr1,20144,196,42,7167,244,211,17,40,GCC,2.394558,0.487562,NaN,32094,0.487705


In [57]:
SNVCalls_PCSI0072.head()

,CHROM,POS,END,X2399011,X2642567,X2642569,X2701060,X2701064,X2810931,X2811120,X2811122,X2811125,X2811128,X2811399,X2812916,X2813255,X2813267,X2813345,X2813387,X2813467,
0,chr1,10179,10180,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...
1,chr1,10351,10352,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...
2,chr1,16256,16257,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...
3,chr1,17593,17594,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...
4,chr1,20143,20144,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...


In [58]:
np.unique(GenomicFeatures_PCSI0072['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [59]:
PCSI0072 = pd.merge(GenomicFeatures_PCSI0072, SNVCalls_PCSI0072,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [60]:
PCSI0072.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399011,X2642567,X2642569,
0,chr1,10180,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793,10179,10180,0,0,0,...
1,chr1,10352,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667,10351,10352,0,0,0,...
2,chr1,16257,69,32,3047,106,76,16,67,CCC,2.333333,0.517413,ncRNA_intronic,35981,0.556604,16256,16257,0,0,0,...
3,chr1,17594,95,19,3588,120,153,0,60,CCT,2.605634,0.631841,ncRNA_intronic,34644,0.500000,17593,17594,0,0,0,...
4,chr1,20144,196,42,7167,244,211,17,40,GCC,2.394558,0.487562,NaN,32094,0.487705,20143,20144,0,0,0,...


In [61]:
PCSI0072_X = PCSI0072.iloc[:,2:15] #Exclude CHROM, POS
PCSI0072_Y = PCSI0072.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [62]:
print PCSI0072_X.shape
print PCSI0072_Y.shape

(83808, 13)
(83808, 133)


In [63]:
PCSI0072_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793
1,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667


In [64]:
PCSI0072_Y[:2]

,X2399011,X2642567,X2642569,X2701060,X2701064,X2810931,X2811120,X2811122,X2811125,X2811128,X2811399,X2812916,X2813255,X2813267,X2813345,X2813387,X2813467,X2813771,X2813775,X2813778,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...


In [65]:
totalSample = PCSI0072_Y.shape[0]
PCSI0072_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print PCSI0072_Y_Val.shape

83808
(83808, 1)


In [66]:
PCSI0072_Y.shape

(83808, 133)

In [67]:
PCSI0072_Y.values[0].shape

(133,)

In [68]:
for i in range(totalSample):
    PCSI0072_Y_Val[i] = np.argmax(np.bincount(PCSI0072_Y.values[i]))

In [69]:
PCSI0072_Y_Val_1 = pd.DataFrame(PCSI0072_Y_Val)
PCSI0072_Y_Val_1.columns = ['Label']

In [70]:
PCSI0072_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [71]:
PCSI0072_dataframe = pd.concat([PCSI0072_X, PCSI0072_Y_Val_1],axis=1)

In [72]:
PCSI0072_dataframe.shape

(83808, 14)

In [73]:
PCSI0072_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793,0
1,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667,0


In [28]:
PCSI0072_dataframe.to_csv('PCSI0072.csv')

In [30]:
#Now we have variable importance ...lets do analysis

In [29]:
%load_ext rpy2.ipython

In [30]:
%%R
# Read in the data
PCSI0072Frame = read.csv("PCSI0072.csv")
str(PCSI0072Frame)

'data.frame':	83808 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  141 133 69 95 196 45 44 24 64 40 ...
 $ NonRefAllele   : int  43 23 32 19 42 4 24 18 12 13 ...
 $ BaseQual       : int  4819 3037 3047 3588 7167 1572 2084 1297 2392 1516 ...
 $ TumourCoverage : int  193 168 106 120 244 49 69 43 76 54 ...
 $ NormalCoverage : int  383 383 76 153 211 20 24 23 48 43 ...
 $ MapQual        : num  17 20 16 0 17 20 12 19 20 37 ...
 $ ReadPosition   : num  47 21 67 60 40 55 51 56 61.5 61 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 13 13 10 12 18 25 26 24 2 3 ...
 $ HomopolymerRate: num  4.9 5.62 2.33 2.61 2.39 ...
 $ GCcontent      : num  0.498 0.537 0.517 0.632 0.488 ...
 $ GenomicLocation: Factor w/ 14 levels "","downstream",..: 1 1 8 8 1 1 5 5 5 5 ...
 $ DistanceSNP    : int  42058 41886 35981 34644 32094 489 6628 10539 12689 3405 ...
 $ StrandBias     : num  0.28 0.167 0.557 0.5 0.488 ...
 $ Label          : num  0 0 0 0

In [31]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(PCSI0072Frame)) 

[1] 0


In [34]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [32]:
%%R
sum(is.na(PCSI0072Frame))  #No NA value now

[1] 0


In [33]:
%%R
write.csv(PCSI0072Frame, file = "PCSI0072_imputed.csv")  #Cleaned CSV without NA

In [34]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 29 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.56 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [77]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/PCSI0072_imputed.csv"
PCSI0072.hex          = h2o.importFile(path = csvPath, destination_frame = "PCSI0072.hex") 
train                 = as.data.frame(PCSI0072.hex)

str(train)

  |======================================================================| 100%
'data.frame':	83808 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  141 133 69 95 196 45 44 24 64 40 ...
 $ NonRefAllele   : int  43 23 32 19 42 4 24 18 12 13 ...
 $ BaseQual       : int  4819 3037 3047 3588 7167 1572 2084 1297 2392 1516 ...
 $ TumourCoverage : int  193 168 106 120 244 49 69 43 76 54 ...
 $ NormalCoverage : int  383 383 76 153 211 20 24 23 48 43 ...
 $ MapQual        : num  17 20 16 0 17 20 12 19 20 37 ...
 $ ReadPosition   : num  47 21 67 60 40 55 51 56 61.5 61 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 13 13 10 12 18 25 26 24 2 3 ...
 $ HomopolymerRate: num  4.9 5.62 2.33 2.61 2.39 ...
 $ GCcontent      : num  0.498 0.537 0.517 0.632 0.488 ...
 $ GenomicLocation: Factor w/ 13 levels "downstream","exonic",..: NA NA 7 7 NA NA 4 4 4 4 ...
 $ DistanceSNP    : int  42058 41

In [78]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [79]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [80]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1 NormalCoverage            1.000000          1.000000   0.055532
2        MapQual            0.931967          0.931967   0.051754
3   NonRefAllele            0.884391          0.884391   0.049112
4      RefAllele            0.705659          0.705659   0.039187
5       BaseQual            0.539393          0.539393   0.029954

---
                    variable relative_importance scaled_importance percentage
53         Trinucleotide.TTT            0.164587          0.164587   0.009140
54         Trinucleotide.TCA            0.156205          0.156205   0.008674
55         Trinucleotide.GCA            0.151291          0.151291   0.008402
56                 GCcontent            0.086329          0.086329   0.004794
57                StrandBias            0.071138          0.071138   0.003950
58 Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [81]:
%%R
write.table(var_importance, file="PCSI0072_variable_importance.csv")

In [82]:
var_importance = pd.read_csv(open('PCSI0072_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [83]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NormalCoverage,1.000000,1.000000,0.055532
2,MapQual,0.931967,0.931967,0.051754
3,NonRefAllele,0.884391,0.884391,0.049112
4,RefAllele,0.705659,0.705659,0.039187
5,BaseQual,0.539393,0.539393,0.029954


In [84]:
GenomicFeatures_PCSI0072[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10180,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793
1,chr1,10352,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667
2,chr1,16257,69,32,3047,106,76,16,67,CCC,2.333333,0.517413,ncRNA_intronic,35981,0.556604
3,chr1,17594,95,19,3588,120,153,0,60,CCT,2.605634,0.631841,ncRNA_intronic,34644,0.500000
4,chr1,20144,196,42,7167,244,211,17,40,GCC,2.394558,0.487562,NaN,32094,0.487705


In [85]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.539392650127411,
 'DistanceSNP': 0.30066937208175698,
 'GCcontent': 0.086329311132430989,
 'GenomicLocation.UTR3': 0.30057114362716703,
 'GenomicLocation.UTR5': 0.30757862329483004,
 'GenomicLocation.downstream': 0.30733063817024203,
 'GenomicLocation.exonic': 0.29277893900871305,
 'GenomicLocation.exonic;splicing': 0.31011497974395802,
 'GenomicLocation.intergenic': 0.17532567679882002,
 'GenomicLocation.intronic': 0.18094779551029197,
 'GenomicLocation.missing(NA)': 0.24602952599525499,
 'GenomicLocation.ncRNA_UTR3': 0.21987646818161,
 'GenomicLocation.ncRNA_exonic': 0.306052356958389,
 'GenomicLocation.ncRNA_intronic': 0.33912152051925704,
 'GenomicLocation.splicing': 0.31590369343757602,
 'GenomicLocation.upstream': 0.28076151013374301,
 'GenomicLocation.upstream;downstream': 0.34965798258781394,
 'HomopolymerRate': 0.40177556872367898,
 'MapQual': 0.93196684122085605,
 'NonRefAllele': 0.8843907117843629,
 'NormalCoverage': 1.0,
 'ReadPosition': 0.182333409786224,
 '

In [86]:
GenomicFeatures_PCSI0072.columns

Index([u'CHROM', u'POS', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'DistanceSNP', u'StrandBias'], dtype='object')

In [87]:
GenomicFeatures_PCSI0072.dtypes

CHROM               object
POS                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
DistanceSNP          int64
StrandBias         float64
dtype: object

In [88]:
GenomicFeatures_PCSI0072.iloc[0,13]

42058

In [89]:
GenomicFeatures_PCSI0072[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10180,141,43,4819,193,383,17,47,CTA,4.897959,0.497512,NaN,42058,0.279793
1,chr1,10352,133,23,3037,168,383,20,21,CTA,5.623656,0.537313,NaN,41886,0.166667


In [90]:
noOfSample = GenomicFeatures_PCSI0072.shape[0]

PCSI0072_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_PCSI0072.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_PCSI0072.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_PCSI0072.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_PCSI0072.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_PCSI0072.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_PCSI0072.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_PCSI0072.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_PCSI0072.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_PCSI0072.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_PCSI0072.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_PCSI0072.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_PCSI0072.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_PCSI0072.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_PCSI0072.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_PCSI0072.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_PCSI0072.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    PCSI0072_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [91]:
#Sorting dictionary
PCSI0072_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in PCSI0072_Dict.items()],reverse=True)]

In [92]:
PCSI0072_Dict_Frame = pd.DataFrame(PCSI0072_Dict1, columns=['POS', 'Value'])

In [93]:
PCSI0072_Dict_Frame.to_csv("PCSI0072_Dict_Frame.csv")

In [94]:
PCSI0072_Dict_Frame = pd.read_csv("PCSI0072_Dict_Frame.csv")

In [95]:
PCSI0072_Dict_Frame[:10]

,Unnamed: 0,POS,Value
0,0,58883659,9179216.631793
1,1,58866011,9127153.465830
2,2,58862718,9122670.614667
3,3,58997006,9119838.337494
4,4,58997003,9119250.618114
5,5,58997017,9117302.044931
6,6,58997030,9111853.218646
7,7,58985571,9100691.937100
8,8,58982822,9097376.209808
9,9,58982814,9096419.591402


In [96]:
GenomicFeatures_PCSI0072_1 = pd.concat([PCSI0072.iloc[:,0:16], PCSI0072_Y_Val_1],axis=1)

In [98]:
rows = PCSI0072_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_PCSI0072_Top20 = GenomicFeatures_PCSI0072_1.loc[GenomicFeatures_PCSI0072_1['POS_x'].isin(rows)]

[58883659 58866011 58862718 58997006 58997003 58997017 58997030 58985571
 58982822 58982814 58978582 58978574 59020363 59031449 59025184 59027650
 59020168 59032042 59022082 59023618]


In [ ]:
GenomicFeatures_PCSI0072_Top20[:10]

In [57]:
GenomicFeatures_PCSI0072_Top20.to_csv('GenomicFeatures_PCSI0072_Top20.csv')